Dados: https://www.kaggle.com/c/whats-cooking/data


Script em python para transformar os dados json em csv:

```python
import json
import codecs
import sys

def load_data(path):
    data = None
    with open(path,"r") as f:
        data = json.load(f)
    return data

def write_data(path, data, istest):
    with codecs.open(path,"w","utf-8") as f:
        f.write("id|"+ ("cuisine|" if not istest else "") +"ingredients")
    
        for line in data:
            f.write("\n")
            row = str(line["id"]) + "|" + \
                    (line["cuisine"] + "|" if not istest else "")
            for ingredient in line['ingredients']:
                row += ingredient + ","
            row = row[0:-1]
            f.write(row)

data = load_data("train.json")
write_data("train.csv",data,False)
data = load_data("test.json")
write_data("test.csv", data, True)
```

In [1]:
data <- read.csv("train.csv", sep = "|")
str(data)

'data.frame':	39774 obs. of  3 variables:
 $ id         : int  10259 25693 20130 22213 13162 6602 42779 3735 16903 12734 ...
 $ cuisine    : Factor w/ 20 levels "brazilian","british",..: 7 17 5 8 8 11 18 10 14 10 ...
 $ ingredients: Factor w/ 39674 levels "(    oz.) tomato sauce,diced tomatoes,chopped onion,dried oregano,red beans,chopped celery,green chilies,ground"| __truncated__,..: 28492 26710 10343 38377 2528 26725 24040 33076 23888 6522 ...


Primeiro, vamos separar cada ingrediente

In [2]:
install.packages('devtools')
library('devtools')
slam_url <- "https://cran.r-project.org/src/contrib/Archive/slam/slam_0.1-37.tar.gz"
install_url(slam_url)
install.packages('tm')

Installing package into ‘/home/carlos/R/x86_64-pc-linux-gnu-library/3.4’
(as ‘lib’ is unspecified)
Installing slam
'/usr/lib/R/bin/R' --no-site-file --no-environ --no-save --no-restore --quiet  \
  CMD INSTALL '/tmp/RtmpoJEDGG/devtools5ee85e102f60/slam'  \
  --library='/home/carlos/R/x86_64-pc-linux-gnu-library/3.4' --install-tests 

Installing package into ‘/home/carlos/R/x86_64-pc-linux-gnu-library/3.4’
(as ‘lib’ is unspecified)


In [2]:
library('tm')

Loading required package: NLP


In [3]:
data$ingredients <- gsub(' ','-',data$ingredients)
data$ingredients <- gsub(',',' ',data$ingredients)

In [4]:
ingred_corpus <- VCorpus(VectorSource(data$ingredients))
print(ingred_corpus)
print("------------")
inspect(ingred_corpus[1:2])
print("------------")
as.character(ingred_corpus[[1]])

<<VCorpus>>
Metadata:  corpus specific: 0, document level (indexed): 0
Content:  documents: 39774
[1] "------------"
<<VCorpus>>
Metadata:  corpus specific: 0, document level (indexed): 0
Content:  documents: 2

[[1]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 116

[[2]]
<<PlainTextDocument>>
Metadata:  7
Content:  chars: 121

[1] "------------"


[1] "romaine-lettuce black-olives grape-tomatoes garlic pepper purple-onion seasoning garbanzo-beans feta-cheese-crumbles"

In [5]:
ingred_dtm <- DocumentTermMatrix(ingred_corpus)

In [6]:
upper_bound <- floor(0.75 * nrow(data))

ingred_dtm_train <- ingred_dtm[1:upper_bound, ]
ingred_dtm_test <- ingred_dtm[(upper_bound+1):nrow(data), ]

train_labels <- data[1:upper_bound, ]$cuisine
test_labels <- data[(upper_bound+1):nrow(data), ]$cuisine

In [7]:
prop.table(table(train_labels))
prop.table(table(test_labels))

train_labels
   brazilian      british cajun_creole      chinese     filipino       french 
  0.01146497   0.01994636   0.03821656   0.06627556   0.01900771   0.06580624 
       greek       indian        irish      italian     jamaican     japanese 
  0.02913175   0.07539390   0.01642642   0.19849145   0.01354341   0.03613812 
      korean      mexican     moroccan      russian  southern_us      spanish 
  0.02095206   0.16195106   0.02058331   0.01287295   0.10935300   0.02510895 
        thai   vietnamese 
  0.03851827   0.02081797 

test_labels
   brazilian      british cajun_creole      chinese     filipino       french 
  0.01257039   0.02101770   0.04082864   0.06999195   0.01890587   0.06868463 
       greek       indian        irish      italian     jamaican     japanese 
  0.03077233   0.07582462   0.01779968   0.19277957   0.01226870   0.03469429 
      korean      mexican     moroccan      russian  southern_us      spanish 
  0.02061545   0.16160499   0.02081657   0.01055913   0.10639582   0.02413516 
        thai   vietnamese 
  0.03921963   0.02051488 

In [10]:
ingred_freq_terms <- findFreqTerms(ingred_dtm_train)
str(ingred_freq_terms)

 chr [1:6706] "-1-inch-thick" "-cook-and-drain" "-cooked-and-drained" ...


In [26]:
ingred_dtm_freq_train <- ingred_dtm_train[, ingred_freq_terms]
ingred_dtm_freq_test <- ingred_dtm_test[, ingred_freq_terms]

In [8]:
convert_counts <- function(x) {
    x <- ifelse(x > 0, "Yes", "No")
}

In [9]:
ingred_train <- apply(ingred_dtm_train, MARGIN = 2, convert_counts)
ingred_test <- apply(ingred_dtm_test, MARGIN = 2, convert_counts)

In [14]:
install.packages("e1071", quiet = TRUE)

Warning message in system2(cmd0, args, env = env, stdout = outfile, stderr = outfile):
“system call failed: Cannot allocate memory”Warning message in install.packages("e1071", quiet = TRUE):
“installation of package ‘e1071’ had non-zero exit status”

In [10]:
library("e1071")

In [11]:
ingred_classifier <- naiveBayes(ingred_train, train_labels)

In [12]:
ingred_test_pred <- predict(ingred_classifier, ingred_test)

ERROR: Error in `[.default`(object$tables[[v]], , nd + islogical[attribs[v]]): subscript out of bounds
